# Example Electrolyzer Usage

This notebook demonstrates how to set up a working electrolyzer example based on a
heavily modified version of the COREWIND *in situ* model.

In [1]:
import pandas as pd

from wombat import Simulation, load_yaml
from wombat.core.library import COREWIND

## Defining the simulation

This example will be a fictionalized offshore electrolyzer in Morro Bay with a single
15 MW turbine and substation, and a 1 MW PEM Electrolyzer. We will also model a single
CTV for servicing at the site.

### Primary configuration

First, we will set up the primary inputs for the model by loading the original inputs,
then modifying the necessary fields, or creating any placeholders for later sections.

In [2]:
config = load_yaml(COREWIND / "project/config", "morro_bay_in_situ.yaml")

config["name"] = "PEM Test"
config["service_equipment"] = ["ctv"]
config["project_capacity"] = 15

config["turbines"] = {}
config["substations"] = {}
config["cables"] = {}
config["electrolyzers"] = {}
config["vessels"] = {}

### Customizing the layout

Since a barebones site is being modeled in this example, we will create a custom data set
by dropping all the substations and turbines, except for the required one of each.
Additionally, we will have to update a the "string" and "order" columns to ensure they
comply with the 0-indexing that WOMBAT expects

It's worth noting that we are using the "_consolidated" version of the layout so that we
can later embed the turbine and substation configurations in the primary configuration
instead of creating a new data in this example.

We wil also be inputting the electrolyzer information in the row for the substation "SS1"
as it already connects to the substation "SS2", b

In [3]:
layout = pd.read_csv(COREWIND / "project/plant/morro_bay_9D_layout_consolidated.csv")
layout = layout.loc[layout.id.isin(("SS1", "SS2", "WTG_0400"))].reset_index(drop=True)

layout

,id,substation_id,name,type,x,y,easting,northing,longitude,latitude,string,order,distance,subassembly,upstream_cable,upstream_cable_name
0,SS1,SS2,SS1,substation,0,4320,623080.06,3918930.21,-121.644512,35.406164,9,0,NaN,corewind_substation,corewind_export,EXP00
1,SS2,SS2,SS2,substation,0,-4320,623080.06,3910290.21,-121.645813,35.328280,9,1,NaN,corewind_substation,corewind_export,EXP01
2,WTG_0400,SS2,WTG_0400,turbine,-2160,-1080,620920.06,3913530.21,-121.669094,35.357751,4,0,NaN,corewind_15MW,corewind_array,ARR40


In [4]:
# Update the string data to ensure compliance with expected 0-indexing
layout.string = [1, 1, 0]
layout.order = [0, 0, 0]

fields = ["ELC1", "ELC1", "electrolyzer", "electrolyzer_sample"]
cols = ["id", "name", "type", "subassembly"]
layout.loc[layout.id.eq("SS1"), cols] = fields

# Replace the original layout file with the updated layout DataFrame
config["layout"] = layout

layout

,id,substation_id,name,type,x,y,easting,northing,longitude,latitude,string,order,distance,subassembly,upstream_cable,upstream_cable_name
0,ELC1,SS2,ELC1,electrolyzer,0,4320,623080.06,3918930.21,-121.644512,35.406164,1,0,NaN,electrolyzer_sample,corewind_export,EXP00
1,SS2,SS2,SS2,substation,0,-4320,623080.06,3910290.21,-121.645813,35.328280,1,0,NaN,corewind_substation,corewind_export,EXP01
2,WTG_0400,SS2,WTG_0400,turbine,-2160,-1080,620920.06,3913530.21,-121.669094,35.357751,0,0,NaN,corewind_15MW,corewind_array,ARR40


### Creating the turbine, substation, and cable models

Here, we will create a turbine, substation, and cable configuration that will have no
failures or mainteance events for the purpose of highlighting the electrolyzer. While
there are no events in the model, placeholders are still required to initialize a
simulation.

In [5]:
no_fail_maintenance = [
    {
        "description": "n/a",
        "time": 0,
        "materials": 0,
        "service_equipment": "CTV",
        "frequency": 0,
    }
]
no_fail_failures = [
    {
        "description": "n/a",
        "time": 0,
        "materials": 0,
        "service_equipment": "CTV",
        "scale": 0,
        "shape": 0,
        "operation_reduction": 0,
        "level": 1,
    }
]

turbine = {
    "capacity_kw": 15000,
    "capex_kw": 1300,
    "power_curve": {
        "file": "2020ATB_NREL_Reference_15MW_240.csv",
        "bin_width": 0.25,
    },
    "turbine": {
        "name": "all subassemblies",
        "maintenance": no_fail_maintenance,
        "failures": no_fail_failures,
    },
}

substation = {
    "capacity_kw": 600,
    "capex_kw": 120,
    "substation": {
        "name": "all subassemblies",
        "maintenance": no_fail_maintenance,
        "failures": no_fail_failures,
    },
}
substation["substation"] = turbine["turbine"]

no_fail_array_cable = {
    "name": "array cable",
    "maintenance": no_fail_maintenance,
    "failures": no_fail_failures,
}
no_fail_export_cable = {
    "name": "export cable",
    "maintenance": no_fail_maintenance,
    "failures": no_fail_failures,
}

Now, we will add the above configurations to the configuration file, ensuring that the
dictionary key matches the input to the "subassembly" or "upstream_cable" column.

In [6]:
config["turbines"]["corewind_15MW"] = turbine
config["substations"]["corewind_substation"] = substation
config["cables"]["corewind_array"] = no_fail_array_cable
config["cables"]["corewind_export"] = no_fail_export_cable

### Vessel configuration

In this step, we will load and insert the CTV into the primary configuration without
any modification.

In [7]:
ctv = load_yaml(COREWIND / "vessels", "ctv1.yaml")

config["vessels"]["ctv"] = ctv

### Create the electrolyzer configuration

Now we will create a fabricated electrolyzer example configuration (no publicly
available cost and failure data to use). It will use a linear efficiency power curve
and be assumed to be a single, 1 MW stack PEM electrolyzer.

To model the power curve using a polynomial efficiency, we would replace
`efficiency_rate` with `p1`, `p2`, `p3`, `p4`, and `p5` for the power to current
conversion. For more information on the electrolyzer inputs, please the explanation in
the `how_to.ipynb` example, or the documentation site version at
https://wisdem.github.io/WOMBAT/examples/how_to.html#the-system-models.

In [8]:
config["electrolyzers"]["electrolyzer_sample"] = {
    "stack_capacity_kw": 1000,
    "capex_kw": 800,
    "n_stacks": 1,
    "power_curve": {
        "efficiency_rate": 39.44,
        "FE": 0.9999999,
        "n_cells": 135,
        "turndown_ratio": 0.1,
    },
    "cathode": {
        "name": "Cathode",
        "maintenance": [
            {
                "description": "annual cathode maintenance",
                "service_equipment": "CTV",
                "time": 12,
                "materials": 6000,
                "frequency": 1,
                "frequency_basis": "years",
            }
        ],
        "failures": no_fail_failures,
    },
    "cooling_system": {
        "name": "cooling",
        "maintenance": [
            {
                "description": "annual anode maintenance",
                "service_equipment": "CTV",
                "time": 6,
                "materials": 1000,
                "frequency": 1,
                "frequency_basis": "years",
            }
        ],
        "failures": no_fail_failures,
    },
    "anode": {
        "name": "Anode",
        "maintenance": [
            {
                "description": "cathode maintenance",
                "service_equipment": "CTV",
                "time": 48,
                "materials": 6000,
                "frequency": 5,
                "frequency_basis": "years",
            }
        ],
        "failures": no_fail_failures,
    },
    "power_system": {
        "name": "DC Current Transducer",
        "maintenance": no_fail_maintenance,
        "failures": [
            {
                "description": "n/a",
                "time": 100,
                "materials": 100000,
                "service_equipment": "CTV",
                "scale": 5,
                "shape": 0,
                "operation_reduction": 1,
                "level": 2,
            }
        ],
    },
}

## Run the simulation

Now, we can create the `Simulation` using the library path and modified configuration
dictionary, and run a simulation.

Note, that it's best to delete the logging files once completed. In this example we use
`sim.run(delete_logs=True, save_metrics_inputs=False)` to ensure that al the created
logging and output data deleted from the machine post-simulation.

In [9]:
sim = Simulation(COREWIND, config)

In [11]:
sim.run(delete_logs=True, save_metrics_inputs=False)

## Check the results

In [12]:
metrics = sim.metrics

In [13]:
metrics.opex("project", by_category=True).style.format("${:,.2f}")

,operations,port_fees,equipment_cost,total_labor_cost,materials_cost,OpEx
0,$0.00,$0.00,"$25,562,401.51","$4,802,371.85","$151,000.00","$30,515,773.36"


In [14]:
metrics.time_based_availability("project", "electrolyzer").style.format("{:.1%}")

,ELC1
time_availability,97.8%


In [15]:
metrics.production_based_availability("project", "electrolyzer").style.format("{:.1%}")

,ELC1
energy_availability,97.8%


In [16]:
metrics.capacity_factor("net", "project", by="electrolyzer").style.format("{:.1%}")

,ELC1
net_capacity_factor,83.4%


In [17]:
metrics.capacity_factor("gross", "project", by="electrolyzer").style.format("{:.1%}")

,ELC1
gross_capacity_factor,85.3%


In [18]:
metrics.h2_production("project", by="total", units="tph").style.format("{:,.1f}")

,total
Project H2 Production (tonnes/hr),"3,708.6"
